In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "3"
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from torch import nn, optim
from transformers import AutoTokenizer
import pytorch_lightning as pl
from pytorch_lightning import seed_everything
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from collections import defaultdict, Counter
from model import Classifier
from data import infer_preprocess
import torch

/home/zelin/.conda/envs/py37-torch/lib/python3.7/site-packages/torchaudio/backend/utils.py:54: UserWarning: "sox" backend is being deprecated. The default backend will be changed to "sox_io" backend in 0.8.0 and "sox" backend will be removed in 0.9.0. Please migrate to "sox_io" backend. Please refer to https://github.com/pytorch/audio/issues/903 for the detail.
  '"sox" backend is being deprecated. '


In [2]:
clf = Classifier.load_from_checkpoint(
    'xxx/lightning_logs/version_7/checkpoints/epoch=2-step=2750.ckpt'
)
clf.eval()
tokenizer = AutoTokenizer.from_pretrained(clf.model_type)
max_len = clf.hparams.max_len
clf.cuda()
None

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


"accumulation":      1
"bs":                128
"find_lr":           False
"gradient_clip_val": 0.1
"input_dir":         ./dataset_v3_combine
"lr":                0.0001
"max_len":           64
"model_type":        bert-base-uncased
"num_classes":       6
"patience":          4
"threshold":         0.5


# AudioCaps

In [ ]:
all_preds_text0 = np.load('../experiment/cache/total_preds_audiocaps0.npy').tolist()
all_preds_text1 = np.load('../experiment/cache/total_preds_audiocaps1.npy').tolist()

In [9]:
batch = infer_preprocess(tokenizer, all_preds_text0, max_len)
for k, v in batch.items():
    batch[k] = v.cuda()
with torch.no_grad():
    logits = clf(batch)
    probs0 = torch.sigmoid(logits).detach().cpu().numpy()

In [10]:
batch = infer_preprocess(tokenizer, all_preds_text1, max_len)
for k, v in batch.items():
    batch[k] = v.cuda()
with torch.no_grad():
    logits = clf(batch)
    probs1 = torch.sigmoid(logits).detach().cpu().numpy()

In [11]:
np.save('cache/probs0_alltrain_audiocaps.npy', probs0)
np.save('cache/probs1_alltrain_audiocaps.npy', probs1)

In [12]:
probs1.shape

(1671, 6)

# Clotho

In [ ]:
all_preds_text0 = np.load('../experiment/cache/total_preds_clotho0.npy').tolist()
all_preds_text1 = np.load('../experiment/cache/total_preds_clotho1.npy').tolist()

In [5]:
batch = infer_preprocess(tokenizer, all_preds_text0, max_len)
for k, v in batch.items():
    batch[k] = v.cuda()
with torch.no_grad():
    logits = clf(batch)
    probs0 = torch.sigmoid(logits).detach().cpu().numpy()

In [6]:
batch = infer_preprocess(tokenizer, all_preds_text1, max_len)
for k, v in batch.items():
    batch[k] = v.cuda()
with torch.no_grad():
    logits = clf(batch)
    probs1 = torch.sigmoid(logits).detach().cpu().numpy()

In [7]:
np.save('cache/probs0_alltrain_clotho.npy', probs0)
np.save('cache/probs1_alltrain_clotho.npy', probs1)